# Conectando ao Prompt Hub

Podemos conectar nossa aplicação ao Prompt Hub do LangSmith, que nos permitirá testar e iterar nossos prompts dentro do LangSmith, e puxar nossas melhorias diretamente para nossa aplicação.

### Configuração

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # Se você não definir isso, os rastreamentos irão para o projeto Default

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Puxar um prompt do Prompt Hub

Puxe um prompt do Prompt Hub colando o trecho de código da UI.

Vamos ver o que puxamos - note que não obtivemos o modelo, então isso é apenas um StructuredPrompt e não é executável.

In [ ]:
prompt

Legal! Agora vamos hidratar nosso prompt chamando .invoke() com nossas entradas

In [ ]:
hydrated_prompt = prompt.invoke({"question": "Você já é capitão?", "language": "Espanhol"})
hydrated_prompt

E agora vamos passar essas mensagens para o Gemini e ver o que obtemos de volta!

In [ ]:
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat
from langsmith.client import convert_prompt_to_openai_format

# NOTA: Convertemos o prompt hidratado para o formato OpenAI compatível e depois para Gemini
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

call_gemini_chat("gemini-1.5-flash", converted_messages, 0.0)

##### [Extra: Apenas LangChain] Puxando a Configuração do Modelo

Também podemos puxar a configuração do modelo salva como um LangChain RunnableBinding quando usamos `include_model=True`. Isso nos permite executar nosso template de prompt diretamente com a configuração do modelo salva.

In [ ]:
prompt

Teste seu prompt!

In [ ]:
prompt.invoke({"question": "Você já é capitão?", "language": "Espanhol"})

### Puxar um commit específico

Puxe um commit específico do Prompt Hub colando o trecho de código da UI.

Execute este commit!

In [ ]:
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat
from langsmith.client import convert_prompt_to_openai_format

hydrated_prompt = prompt.invoke({"question": "Como é o mundo?", "language": "Português"})
# NOTA: Podemos usar este utilitário do LangSmith para converter nosso prompt hidratado para o formato openai
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

call_gemini_chat("gemini-1.5-flash", converted_messages, 0.0)

### Fazendo Upload de Prompts

Você também pode facilmente atualizar seus prompts no hub programaticamente.

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """Você é um assistente para tarefas de perguntas e respostas.
Use as seguintes partes do contexto recuperado para responder à pergunta mais recente na conversa.

Seus usuários só falam francês, certifique-se de responder aos seus usuários apenas em francês.

Conversa: {conversation}
Contexto: {context} 
Pergunta: {question}
Resposta:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

Você também pode fazer push de um prompt como uma RunnableSequence de um prompt e um modelo. Isso é útil para armazenar a configuração do modelo que você quer usar com este prompt. O provedor deve ser suportado pelo playground do LangSmith.

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_google_genai import ChatGoogleGenerativeAI

client=Client()
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

french_prompt = """Você é um assistente para tarefas de perguntas e respostas.
Use as seguintes partes do contexto recuperado para responder à pergunta mais recente na conversa.

Seus usuários só falam francês, certifique-se de responder aos seus usuários apenas em francês.

Conversa: {conversation}
Contexto: {context} 
Pergunta: {question}
Resposta:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)